# Hands-on session: Variational ground state preparation for the Anderson model

In this notebook, you will get familiar with myqlm, a free library for fermionic simulation. 

You will see how to prepare the ground state of an Anderson impurity Hamiltonian within the Variational Quantum Eigensolver. You will investigate about the structure of the ansatz circuit that is required in both uncorrelated ($U=0$) and correlated ($U=1$) settings. You will play with different optimizers in the presence of shot noise. Finally you'll add some hardware noise model. 

I strongly encourage you to play around with the different objects you will encounter!

In [ ]:
#!pip install myqlm
#!pip install myqlm-simulators

#!python -m qat.magics.install

⚠️
__(Google-colab specific)__
At that point, you need to hit 'Runtime' in the menu bar and click 'Restart runtime' so that you'll be able to display circuits.

In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt

from qat.fermion.hamiltonians import make_anderson_model, ElectronicStructureHamiltonian
from qat.fermion.transforms import transform_to_jw_basis

from qat.lang.AQASM import Program, RY
from qat.plugins import ScipyMinimizePlugin, SeqOptim, MultipleLaunchesAnalyzer

from qat.qpus import get_default_qpu

# toolbox
from qat.fermion.chemistry.ucc import transform_integrals_to_new_basis
from qat.lang.AQASM import RX, RZ, CNOT


## Variational quantum preparation of the ground state
Here we'll prepare the ground state of some embedded model with VQE.

We'll construct the following embedded model Hamiltonian, a single-impurity Anderson model (SIAM):

$H_{\mathrm{SIAM}} = U d^{\dagger}_{\uparrow}d_{\uparrow}d^{\dagger}_{\downarrow}d_{\downarrow}
                    -\mu ( d^{\dagger}_{\uparrow}d_{\uparrow} + d^{\dagger}_{\downarrow}d_{\downarrow})
                       + \sum_{p=1}^{N_b}\sum_{\sigma=\uparrow, \downarrow} (V_p d^{\dagger}_{\sigma} f_{p\sigma} + \mathrm{h.c.}) 
                   + \sum_{p=1}^{N_b}\sum_{\sigma=\uparrow, \downarrow} \epsilon_p f^{\dagger}_{p\sigma} f_{p\sigma} $
                   
where $d/d^{\dagger}$ operators correspond to the correlated, 'impurity' site and the $f_p/f_p^{\dagger}$ operators, to the $N_b$ bath sites. 

We'll consider the simplest case: $N_b=1$ (corresponding to 4 fermionic modes due to spin degeneracy, and thus, 4 qubits).

### 1. $U=0$ case

Let's consider first a non-interacting system. We start by creating a second-quantized hamiltonian, get its matrix and diagonalize it to determine the ground state energy so we'll be able to evaluate how well our VQE procedure performs.

In [ ]:
U = 0
mu = U/2 # half-filling
V = [1]
epsilon = [2]

hamilt_0 = make_anderson_model(U, mu, V, epsilon) 
h_mat_0 = hamilt_0.get_matrix()
eigvals, eigvecs = np.linalg.eigh(h_mat_0)
E0_U0 = eigvals[0]
print('E0 at U=0:', E0_U0)

The object `hamilt` is an instance of `ElectronicStructureHamiltonian`, which is a class aimed at manipulating second-quantized Hamiltonians $H = \sum_{pq} h_{pq}f^{\dagger}_p f_q + \frac{1}{2}\sum_{pqrs} h_{pqrs}f^{\dagger}_p f^{\dagger}_q f_r f_s + c I$.

Its attribues are `hpq` and `hpqrs` and `constant_coeff`. The function `make_anderson_model` creates these objects for us, from the more natural fields $U, \mu, V$ and $\epsilon$.

In [ ]:
print(hamilt_0.hpq)

We need to transform this Hamiltonian onto an qubit observable, namely a spin Hamiltonian, so that we'll be able to measure the energy associated to a given circuit.

We can do this through various fermions-to-spins transforms. Here, we use the Jordan-Wigner method.

In [ ]:
h_spin_0 = transform_to_jw_basis(hamilt_0)

Let's now construct our ansatz circuit! We start by considering what is probably the simplest variational circuit one can think of: a product circuit. It can reach any state of the form $(\alpha_1 |0\rangle + \beta_1|1\rangle)\otimes...\otimes (\alpha_{\mathrm{nbqbits}} |0\rangle + \beta_{\mathrm{nbqbits}}|1\rangle)$.

In [ ]:
def construct_product_circ(nbqbits=4):
    prog = Program()
    reg = prog.qalloc(nbqbits)
    theta = [prog.new_var(float, '\\theta_%s'%i)
                 for i in range(nbqbits)]

    for ind in range(nbqbits):
        RY(theta[ind])(reg[ind])

    circ = prog.to_circ()
    return circ

circ = construct_product_circ()
%qatdisplay circ

We need to define an optimizer that will allow to compute the ansatz' circuits parameters yielding the lowest energy. Here we use standard scipy methods: play with them to find a good one!

In [ ]:
method = 'BFGS' # try different ones! 
scipy_optimizer =  ScipyMinimizePlugin(method=method) # supposes random initialization
qpu = get_default_qpu() # default noise-free emulator
stack = scipy_optimizer | qpu # note the structure

job = circ.to_job(observable=h_spin_0) 

Ok, everything is set, let's run our VQE algorithm...

In [ ]:
res = stack.submit(job)
print(res.value)

We can easily visualize the VQE convergence, as the different energy evaluations are stored in the `meta_data` of the result.

In [ ]:
VQE_trace = res.meta_data['optimization_trace']
plt.axhline(E0_U0, ls='dashed', label='$E_0$', color='black', lw=3)
plt.plot(eval(VQE_trace), label='VQE energies', lw=3)
plt.grid()
plt.xlabel('optimization step')
plt.ylabel('energy')
plt.legend()

Huh, weird... $U=0$ means the ground state is uncorrelated. A product circuit should do. Can you think about a way to to make the product circuit work?

__Hint__: take a look at the 'toolbox' part of the first cell of the notebook where we make all the imports!

In [ ]:
# Type code here
# ....

Here you'll notice we haven't set any initialization for the circuit's parameters: this implies the initial parameters are drawn randomly. Does the initialization have a strong influence on the value of the converged energy? 

You can have a quick answer to this question easily using the `MultipleLaunchesAnalyzer` plugin. It will perform several VQE runs corresponding to different random initializations, keep the best one as the VQE result but do some book-keeping: look at the `meta_data` of the result!

In [ ]:
multiplier = MultipleLaunchesAnalyzer(n_runs=10)
multiple_launches_stack = multiplier | stack

res = multiple_launches_stack.submit(job)
# inspect res here...
# ...

Before turning to the $U=1$ case, let's make a first step towards realistic simulations by incorporating some shot noise. This is done at the job level. Here we'll use a gradient-free method of scipy (try using BFGS instead if you want to understand why):

In [ ]:
from qat.fermion.hamiltonians import ElectronicStructureHamiltonian

h_spin_0_rotated = ElectronicStructureHamiltonian.load('h_spin_0_rotated')
job_with_shot_noise = circ.to_job(observable=h_spin_0_rotated, nbshots=1000)

scipy_optimizer =  ScipyMinimizePlugin(method='COBYLA') # supposes random initialization
scipy_optimizer_stack = scipy_optimizer | qpu

scipy_res = scipy_optimizer_stack .submit(job_with_shot_noise)

plt.axhline(E0_U0, ls='dashed', label='$E_0$', color='black', lw=3)
plt.plot(eval(scipy_res.meta_data['optimization_trace']), lw=3, label='VQE energy (best run)')
plt.grid()
plt.xlabel('optimization step')
plt.ylabel('energy')
plt.legend()


Run the previous cell several times. Do you obtain similar results?

In the lecture, I have presented a method exhibiting a nice shot noise-resilience, dubbed Rotosolve. Can you check it is indeed more shot-noise resilient than e.g., COBYLA?

__Hint__: Using the `MultipleLaunchesAnalyzer` it is very easy to do a quick statistical analysis!

In [ ]:
seq_optim = SeqOptim(ncycles=10) # ncycles is the number of times the optimizer goes 
                                 # through all of the parameters to update them locally

seq_optimizer_stack = seq_optim | qpu

# ...type here...

Plot the VQE curve obtained with the Rotosolve optimizer. Why does it look funny?

In [ ]:
# ...type here

Enough with uncorrelated states! They're boring: we know how to compute them sub-exponentially with classical computers. Let's deal with the real stuff and set $U$ to 1.

In [ ]:
U = 1
mu = U/2
hamilt_1 = make_anderson_model(U, mu, V, epsilon) 
h_mat_1 = hamilt_1.get_matrix()
eigvals, eigvecs = np.linalg.eigh(h_mat_1)
E0_U1 = eigvals[0]
print('E0 at U=1:', E0_U1)

h_spin_1 = transform_to_jw_basis(hamilt_1)

Now we know that the product ansatz is doomed to failed. Try constructing an ansatz that works! 

__Hint__: How do you generate entanglement? Once again, take a look at the toolbox. You can also look up for 'hardware-efficient circuit' on the internet (note: the wording is slightly misleading, as there isn't a single proper way to build a hardware-efficient circuit).

In [ ]:
# try create an ansatz that works...

def construct_correlated_circ(nbqbits=4, n_layers=3):
    prog = Program()
    reg = prog.qalloc(nbqbits)
    n_parameters = ... # specify which total number of parameters 
                       # your circuit will have
    
    theta = [prog.new_var(float, '\\theta_%s'%i)
                 for i in range(n_parameters)]

    # for instance, start with product circuit
    ind = 0
    for _ in range(nbqbits):
        RY(theta[ind])(reg[ind])
        ind += 1
        
    # build up entanglement
    for _ in range(n_layers):
        ...

        
    circ = prog.to_circ()
    return circ

In [ ]:
# try exploring how adding complexity to your circuit increases your expressivity...
# ...

Once you're satisfied with the expressivity of your circuit, try adding some real physical noise (as opposed to statistical). You can use for that the depolarizing plugin.

In [ ]:
from qat.qpus import PyLinalg
from depolarizing_plugin import DepolarizingPluginVec, make_matrix

qpu = PyLinalg()
depol_plugin = DepolarizingPluginVec(prob_1qb=0.001, prob_2qb=0.01)

depol_qpu = depol_plugin | qpu
depol_scipy_optimizer_stack = scipy_optimizer | depol_plugin | qpu

#job = ...
#res = ...

Now, is adding an entangling layer still systematically beneficial to the performances of the VQE determination of your ground state? If not, what is the optimal number of layers?